In [2]:
import pandas
import duckdb
from datasets import load_dataset

/Users/felix/MSE/03_projects/MT/06_rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [65]:
NAME = 'tiny'
SIZE = 500

WIKI_ALL = f'../data/wikipedia_kilt.duckdb'
OUT_KILT_SUBSET = f'../data/{NAME}_{SIZE}_kilt_fever_train.jsonl'
OUT_WIKI_SUBSET = f'../data/{NAME}_{SIZE}_kilt_wiki.jsonl'

In [66]:
def extract_wikipedia_link(row):

    item = row['output'][0]['provenance'][0]

    return {
        'wikipedia_id': item['wikipedia_id'],
        'section': item['section'],
        'start_paragraph_id': item['start_paragraph_id'],
        'end_paragraph_id': item['end_paragraph_id']
    }

    

In [67]:
kilt_fever = load_dataset("kilt_tasks", name="fever")

In [1]:
kilt_fever = load_dataset("kilt_tasks", name="wiki")

NameError: name 'load_dataset' is not defined

In [68]:
allowed = ['SUPPORTS', 'REFUTES']

train_clean = kilt_fever['train'].filter(lambda row: (row['output'][0]['answer'] in allowed and len(row['output'][0]['provenance']) > 0))
train_clean

Dataset({
    features: ['id', 'input', 'meta', 'output'],
    num_rows: 67076
})

In [69]:
subset = train_clean.select(range(SIZE))

In [70]:
relevant = subset.map(extract_wikipedia_link)
relevant_wiki_pages = relevant['wikipedia_id']

relevant.to_json(OUT_KILT_SUBSET)

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 143.76ba/s]


430213

In [ ]:
relevant.to_sql()

In [71]:
relevant_wiki_pages = subset.map(extract_wikipedia_link)['wikipedia_id']

In [72]:
relevant_wiki_pages[0:-1]

['43386202',
 '1591817',
 '31978361',
 '43376',
 '76383',
 '30339',
 '31186339',
 '46301800',
 '9233381',
 '23041',
 '4155050',
 '1221476',
 '69795',
 '44319609',
 '1383751',
 '31341',
 '27338',
 '104697',
 '11929259',
 '1219461',
 '421292',
 '149709',
 '558139',
 '21980',
 '22941812',
 '33142873',
 '347105',
 '2675554',
 '5769',
 '1475688',
 '3092836',
 '246309',
 '3365103',
 '14814',
 '81425',
 '76033',
 '1722020',
 '17909',
 '138483',
 '3257815',
 '1067262',
 '3969',
 '16642',
 '18119',
 '18724',
 '47019680',
 '316592',
 '37852474',
 '199919',
 '33509716',
 '404194',
 '50605',
 '2557423',
 '10746393',
 '17157119',
 '1645518',
 '711102',
 '247463',
 '26709',
 '480058',
 '36487',
 '82533',
 '235959',
 '18618306',
 '13149',
 '27694404',
 '936829',
 '100730',
 '43373',
 '18060198',
 '376978',
 '40179756',
 '15716827',
 '23087494',
 '60070',
 '2038117',
 '28229246',
 '44219',
 '8980330',
 '144936',
 '9228',
 '2242226',
 '44469',
 '47767310',
 '838950',
 '26658567',
 '27007631',
 '388018'

In [73]:
con = duckdb.connect(WIKI_ALL)
relevant = con.execute("SELECT * FROM wiki WHERE wikipedia_id in ?", [relevant_wiki_pages[0:-1]]).df()
relevant.to_json(OUT_WIKI_SUBSET, orient='records', lines=True)